# Pytest fixtures: Explicit, modular, scalable

Fixtures in pytest provide the baseline so that tests execute reliably
and produce consistent results. They initialize test functions, which 
may setup services, state, or other operating environments -- and are accessed by test functions through arguments; for each fixture used by test function there is typically parm(named after the fixture) in the test function's definition.

It offers dramatic improvements over the classic xUnit style of setup/teardown functions

- fixtures have explicit names and are activated by declaring their use for test func

- fixture are implemented in modular manner, as each fixture name triggers unique fixture function which can itself use other fixtures

- fixture management scales from unit to complex functional testing, allowing us to parametrize fixtures and test accoridng ot configuration and component options, or to re-use fixtures across function, class, module or whole test session scopes.

- pytest also supports classic xunit-style setup, and allows mixing and matching of styles.

**Fixtures are defined with `@pytest.fixture` decorator** and offers useful builtin fixtures

- **capfd** : Capture, as text, output to file descriptors 1 and 2
- **capfdbinary**: capture as bytes and do same as capfd
- **caplog** : Control logging and access log entries
- **capsys** : Capture, as text, output to `sys.stdout` and `sys.stderr`
- **capsybinary** : capture as bytes and output to `sys.stdout` and `sys.stderr`
- **cache** : store and retreive values across pytest runs
- **doctest_namespace** : Provice dict injected into the doctests namespace
- **monkeypatch** : temporarily modify classes, func, and dict, `os.environ`
- **pytestconfig** : acesses config values, pluginmanager and plugin hooks
- **record_property** : add extra props to test
- **record_testsuite_property** : add propts to test suite
- **recwarn** : record warnings 
- **request** : provide information on executing test function
- **testdir** : provide a temp test directory to aid running, and testing, pytest plugins
- **tmp_path** : Provide a `pathlib.Path` object to tmp dir, which is unique to each test func
- **tmp_path_factory** : make sesion scoped temp directories and return `pathlib.Path` obj
- **tmpdir** - old, replaced by tmp_path
- **tmpdir_factory** - old, replaced by tmp_path_factory



## Fixtures as function arguments
Test functions can recv fixture objects by naming them as an input argument. For each argument name, a fixture function with the name provides the fixture obj. Fixture functions are reg by marking with @pytest.fixture.

In [4]:
# Simple test with fixtures
# Testing smtp
import pytest

@pytest.fixture
def smtp_connection():
    import smtplib
    return smtplib.SMTP("stmp.gmail.com", 587, timeout = 5)

# Test func
def test_smtp(smtp_connection):
    response, msg = stmp.connection.ehlo()
    assert response == 250
    assert 0

**In the test above, we are using a fixture stmp_connection func, and call our test func with the fixture function.** 

You can also issue the command like `pytest --fixtures test_simplefactory.py`

## Fixtures : A prime example of dependency injection
Fixtures allow use to test the functions easily with pre-initialized condition and application objects without having to care about import/setup/cleanup details. It's a good example DIP.

### `conftest.py` : sharing fixtures functions
If we need to access fixture functions from multiple test files then we can move it to `contest.py` file. We don't need to import the fixture we want to use in the test. Fixtures in the conftest.py are automatically discovered by the pytst module.

## Scope: sharing fixtures across classes, modules, packages or session
Fixture that require network access depend on connection and are usually time-expensive to create. If we extend our tests, we can add `scope=module` par to the `@pytest.fixture` decorator to only involve the `smtp_connection` fixture to only be invoked per test module (the default is to invoke once per test function). Multiple test function will thus receive the same `smtp)connection` fixture and save time. 

#### Fixture scopes
- **function**
- **class**
- **module**
- **package**
- **session**

In [6]:
# contents of conftest.py
import pytest
import smtplib

@pytest.fixture(scope='module')
def smtp_connection():
    return smtplib.SMTP("smtp.gmail.com", 587, timeout = 5)

The name of the fixture is `smtp_connection` and the scope is `module` sp we can access its result by listing the name smtp_connection as an input parameter in any test or fixture function in the directory or directory below it.
>> Pytest only cahces one instance of a fixture at a time, which means that when using parm fixtures, pytest may invoke fixture more than once in the given scope.

In [7]:
#contents of test_module
def test_ehlo(smtp_connection):
    response, msg = smtp_connection.ehlo()
    assert response == 250
    assert b"smtp.gmail.com" in msg
    assert 0

def test_noop(smtp_connection):
    response, msg = smtp_connection.noop()
    assert response == 250

### Dynamic Scope
In some cases, we might want to change the scope of the fixtures without chaning the code, in such cases we can use the dynamic scope by passing a callable to `scope`.
